# Summary of a NN model from `SMD anomaly detection` paper.

Implementation is [here](https://github.com/DongYuls/SMD_Anomaly_Detection) and paper is [here](https://www.mdpi.com/1424-8220/18/5/1308).  
The problem being solved is the anomaly detection based on environment noise. In particular, authors uses sound produced by Surface-Mounted Device machine (SMD).  
Authors propose the following approach. It's hard to work with sound in time domain. But going to a frequency domain may be a benefit. Thus, input to an anomaly detection model is a spectrogram, or in other words, an image. Authors apply STFT (short-time Fourier transform) with window size 2048 and stride equal to 512 points.  
> Sound time series -> STFT -> Spectrogram -> Image -> CNN-based autoencoder.

**Model summary**:  
(batch size = 1, FLOPs = approximate number of Multiply-Add operations)
1. Forward:   7.32 gFLOPs  
2. Backward: 14.64 gFLOPs
3. Training: 21.96 gFLOPs
4. Number of parameters: 4,812,288 (~ 18.3 MB)

The reason for this complexity is deep CNN autoencoder (some layers are not shown):
![SMD anomaly detection model (some layers are not shown)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5982511/bin/sensors-18-01308-g005.jpg)

In [1]:
import os, sys
import pandas as pd
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('')), 'python'))
from nns.nns import ModelSummary, reset_keras, printable_dataframe, estimate
from nns.model import Model
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Activation

In [2]:
class SMDAnomalyDetection(Model):
    """ 
    https://github.com/DongYuls/SMD_Anomaly_Detection/blob/master/model.py
    TODO: refactor conv_block and conv_transpose_block.
    """
    def __init__(self):
        super().__init__('SMDAnomalyDetection')

    @staticmethod
    def conv_block(input_, num_filters, kernel, strides=(1, 1), padding='same', activation='relu', name=None, bn=True):
        x = Conv2D(num_filters, kernel, strides=strides, padding=padding, use_bias=False, name=name + '/conv')(input_)
        if bn:
            x = BatchNormalization(scale=False, name=name + '/bn')(x)
        if activation is not None:
            x = Activation(activation=activation, name=name + '/' + activation)(x)
        return x

    @staticmethod
    def conv_transpose_block(input_, num_filters, kernel, strides=(1, 1), padding='same', activation='relu', name=None, bn=True):
        x = Conv2DTranspose(num_filters, kernel, strides=strides, padding=padding, use_bias=False, name=name + '/convt')(input_)
        if bn:
            x = BatchNormalization(scale=False, name=name + '/bn')(x)
        if activation is not None:
            x = Activation(activation=activation, name=name + '/' + activation)(x)
        return x

    def create(self):
        input_ = Model.Input((1024, 32, 1))
        x = input_
        print("Input shape: {}".format(str(x.shape)))
        # Encoder
        ConvBlock = SMDAnomalyDetection.conv_block
        for idx, num_channels in enumerate([64, 64, 96, 96, 128]):
            x = ConvBlock(x, num_channels, (5, 5), (2, 1), name='enc/conv0{}'.format(idx+1))
        for idx, num_channels in enumerate([128, 160, 160]):
            x = ConvBlock(x, num_channels, (4, 4), (2, 2), name='enc/conv0{}'.format(idx+6))
        x = ConvBlock(x, 192, 3, (2, 2), name='enc/conv09')
        x = ConvBlock(x, 192, 3, (2, 2), name='enc/conv10', activation=None, bn=False)
        print("Code shape: {}".format(str(x.shape)))

        # Decoder
        ConvTBlock = SMDAnomalyDetection.conv_transpose_block
        x = ConvTBlock(x, 192, (3, 3), (2, 2), name='dec/convt01')
        x = ConvTBlock(x, 160, (3, 3), (2, 2), name='dec/convt02')
        for idx, num_channels in enumerate([160, 128, 128]):
            x = ConvTBlock(x, num_channels, (4, 4), (2, 2), name='dec/convt0{}'.format(idx+3))
        for idx, num_channels in enumerate([96, 96, 64, 64]):
            x = ConvTBlock(x, num_channels, (5, 5), (2, 1), name='dec/convt0{}'.format(idx+6))
        x = ConvTBlock(x, 1, (5, 5), (2, 1), name='dec/conv10'.format(idx+6), bn=False, activation=None)
        print("Output shape: {}".format(str(x.shape)))

        return models.Model(input_, x, name=self.name)

In [3]:
perf = []
estimate(SMDAnomalyDetection(), perf, perf)

Input shape: (None, 1024, 32, 1)
Code shape: (None, 1, 1, 192)
Output shape: (None, 1024, 32, 1)
Layer not recognized (type=<class 'tensorflow.python.keras.engine.input_layer.InputLayer'>, name=input)


,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(1024, 32, 1)",0.000000,0,32768,0.000000,0.131072
1,enc/conv01/conv,"(512, 32, 64)",0.026214,1600,1048576,0.006400,4.194304
2,enc/conv01/bn,"(512, 32, 64)",0.000000,128,1048576,0.000512,4.194304
3,enc/conv01/relu,"(512, 32, 64)",0.000000,0,1048576,0.000000,4.194304
4,enc/conv02/conv,"(256, 32, 64)",0.838861,102400,524288,0.409600,2.097152
5,enc/conv02/bn,"(256, 32, 64)",0.000000,128,524288,0.000512,2.097152
6,enc/conv02/relu,"(256, 32, 64)",0.000000,0,524288,0.000000,2.097152
7,enc/conv03/conv,"(128, 32, 96)",0.629146,153600,393216,0.614400,1.572864
8,enc/conv03/bn,"(128, 32, 96)",0.000000,192,393216,0.000768,1.572864
9,enc/conv03/relu,"(128, 32, 96)",0.000000,0,393216,0.000000,1.572864


In [5]:
column_oder = ['name', 'phase', 'input_shape', 'num_parameters', 'param_memory', 'flops', 'activation_memory']
columns = {'name': 'Model', 'phase': 'Phase', 'input_shape': 'Input shape', 'num_parameters': '#Parameters',
           'param_memory': 'Model size (MB) FP32', 'flops': 'GFLOPs (multiply-add)',
           'activation_memory': 'Activation size (MB) FP32'}
df = pd.DataFrame(perf, columns=column_oder)
df.rename(columns=columns)

,Model,Phase,Input shape,#Parameters,Model size (MB) FP32,GFLOPs (multiply-add),Activation size (MB) FP32
0,SMDAnomalyDetection,inference,"(1024, 32, 1)",4812288,19.249152,6.676476,56.425216
1,SMDAnomalyDetection,training,"(1024, 32, 1)",4812288,19.249152,20.029428,112.850432
